<a href="https://colab.research.google.com/github/idutsu/kirikuchikun-ai/blob/main/src/janome.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# 必要なライブラリをインストール
!apt-get install -y mecab libmecab-dev mecab-ipadic-utf8
!pip install mecab-python3 pandas
!pip install unidic
!python -m unidic download

import os
import csv
import glob
import html
import json
import pandas as pd
import MeCab
from tqdm import tqdm

# MeCabを使ってwikipedia本文から「名詞/を/動詞」のパターンを抽出してCSVに書き込む関数
def extract_noun_particle_verb_mecab(directory, output_csv, max_articles=None):

    tagger = MeCab.Tagger()

    # 既存のCSVから処理済みArticle_IDを取得（文字列型に統一）
    processed_ids = set()
    if os.path.exists(output_csv):
        print("既存のCSVファイルを読み込み中...")
        try:
            df = pd.read_csv(output_csv, usecols=['Article_ID'], dtype={'Article_ID': str})
            processed_ids = set(df['Article_ID'].unique())
            print(f"処理済みID数: {len(processed_ids)}")
        except Exception as e:
            print(f"CSVファイル読み込みエラー: {e}")

    # JSONファイルの検索
    json_files = glob.glob(os.path.join(directory, '**', '*.json'), recursive=True)
    print(f"見つかったJSONファイルの数: {len(json_files)}")

    # CSVに結果を書き込み
    with open(output_csv, 'a', newline='', encoding='utf-8') as csvfile:
        csv_writer = csv.writer(csvfile)
        if os.stat(output_csv).st_size == 0:
            csv_writer.writerow(['Article_ID', 'Title', 'Extracted_Phrase'])

        articles_processed = 0
        for json_file in tqdm(json_files, desc="JSONファイルの処理"):
            try:
                with open(json_file, 'r', encoding='utf-8') as f:
                    for line in f:
                        if max_articles and articles_processed >= max_articles:
                            print("指定された記事数に達しました。処理を終了します。")
                            return

                        article = json.loads(line)
                        article_id = str(article.get('id', ''))
                        if article_id in processed_ids:  # 処理済みIDはスキップ
                            continue

                        title = article.get('title', '')
                        text = html.unescape(article.get('text', ''))

                        # MeCabで形態素解析
                        parsed = tagger.parse(text)
                        tokens = [line.split('\t') for line in parsed.splitlines() if '\t' in line]

                        # トークンリスト作成: (表層形, 品詞1, 基本形) の取得
                        token_list = []
                        for line in tagger.parse(text).splitlines():
                            if not line.strip() or line == 'EOS':  # 空行やEOSをスキップ
                                continue
                            parts = line.split('\t')
                            if len(parts) < 2:  # タブ区切りの要素が足りない場合はスキップ
                                continue

                            surface = parts[0]  # 表層形
                            features = parts[1].split(',')
                            pos1 = features[0]  # 品詞1
                            base_form = features[6] if len(features) > 6 and features[6] != '*' else surface  # 基本形
                            token_list.append((surface, pos1, base_form))

                        # デバッグ: トークンの内容を確認（先頭20個）
                        # print("----- トークン一覧（先頭20個）-----")
                        # for token in token_list[:20]:
                        #     print(token)
                        # print("------------------------")

                        # 名詞 + 助詞「を」 + 動詞 のパターンマッチング
                        for i in range(len(token_list) - 2):
                            first, second, third = token_list[i], token_list[i+1], token_list[i+2]
                            if (first[1] == '名詞' and second[0] == 'を' and third[1] == '動詞'):
                                phrase = first[0] + second[0] + third[0]  # 動詞は基本形ではなく表層形を使用
                                #print(f"マッチしたフレーズ: {phrase}")  # デバッグ出力
                                csv_writer.writerow([article_id, title, phrase])

                        processed_ids.add(article_id)
                        articles_processed += 1

            except Exception as e:
                print(f"ファイル {json_file} の処理中にエラーが発生しました: {e}")

    print(f"処理が完了しました。結果は {output_csv} に保存されています。")

# 関数を実行
input_directory = "/content/drive/MyDrive/kirikuchikun-ai/wikipedia_json_output"  # JSONファイルが格納されているディレクトリ
output_csv = "/content/drive/MyDrive/kirikuchikun-ai/csv/wikipedia-a-to-b.csv"    # 出力CSVの保存先
extract_noun_particle_verb_mecab(input_directory, output_csv)
